# Learnable Boundary Guided Adversarial Training

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '4,5,6,7'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/lbgat/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.1
momentum = 0.9
weight_decay = 2e-4
batch_size = 128
total_epochs = 100
lam = 1
epsilon = 0.031
alpha = 0.007
num_repeats = 10

## Inner maximization

In [3]:
def inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon, epsilon).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    
    for _ in range(num_repeats):
        x.requires_grad_()
        logits = model(x)
        loss = xent(logits, targets)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model_nat, model_rob, dataloader, optimizer, num_classes, 
             lam=1, epsilon=8/255, alpha=2/255, num_repeats=10):
    model_nat.train()
    model_rob.train()
    total = 0
    total_loss = 0
    total_correct_nat = 0
    total_correct_rob = 0
        
    mse = nn.MSELoss()
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        logits_nat = model_nat(inputs)
        x = inner_max(model_rob, xent, inputs, targets, epsilon, alpha, num_repeats)
        logits_rob = model_rob(x)
        
        loss = 0.5*mse(logits_rob, logits_nat) + lam*xent(logits_nat, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct_nat = torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum()
        total_correct_nat += num_correct_nat
        num_correct_rob = torch.argmax(logits_rob.data, dim=1).eq(targets.data).cpu().sum()
        total_correct_rob += num_correct_rob
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc nat: %.4f (avg: %.4f) | acc rob: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), 
                     total_loss/len(dataloader), 
                     num_correct_nat/batch, total_correct_nat/total,
                     num_correct_rob/batch, total_correct_rob/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            with torch.enable_grad():
                x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
            
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
    
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model_nat = nn.DataParallel(get_network(model_type, num_classes).cuda())
model_rob = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD([{'params': model_nat.parameters()},
                       {'params': model_rob.parameters()}], 
                      lr=lr, momentum=momentum, weight_decay=weight_decay)

scheduler = [75, 90]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model_nat, model_rob, train_dataloader, optimizer, num_classes, lam, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model_rob, valid_dataloader, alpha, epsilon, num_repeats)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict_nat': model_nat.state_dict(),
                       'state_dict_rob': model_rob.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 2.4413 (avg: 0.0064) | acc nat: 0.0859 (avg: 0.0859) | acc rob: 0.0000 (avg: 0.0000) |
Epoch 0 [100/383] | loss: 1.8231 (avg: 0.5491) | acc nat: 0.3984 (avg: 0.3147) | acc rob: 0.1484 (avg: 0.1294) |
Epoch 0 [200/383] | loss: 1.6855 (avg: 1.0141) | acc nat: 0.5000 (avg: 0.3830) | acc rob: 0.1875 (avg: 0.1550) |
Epoch 0 [300/383] | loss: 1.6060 (avg: 1.4452) | acc nat: 0.4688 (avg: 0.4254) | acc rob: 0.2656 (avg: 0.1683) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [04:08<00:00,  4.03it/s, acc_nat=0.341, acc_rob=0.2]


Epoch 1 [0/383] | loss: 1.4902 (avg: 0.0039) | acc nat: 0.5781 (avg: 0.5781) | acc rob: 0.1797 (avg: 0.1797) |
Epoch 1 [100/383] | loss: 1.4410 (avg: 0.3898) | acc nat: 0.6172 (avg: 0.6076) | acc rob: 0.2109 (avg: 0.2257) |
Epoch 1 [200/383] | loss: 1.3150 (avg: 0.7554) | acc nat: 0.6797 (avg: 0.6278) | acc rob: 0.2578 (avg: 0.2271) |
Epoch 1 [300/383] | loss: 1.3708 (avg: 1.1035) | acc nat: 0.6562 (avg: 0.6429) | acc rob: 0.2656 (avg: 0.2333) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [04:07<00:00,  4.05it/s, acc_nat=0.448, acc_rob=0.243]


Epoch 2 [0/383] | loss: 1.4073 (avg: 0.0037) | acc nat: 0.6719 (avg: 0.6719) | acc rob: 0.2266 (avg: 0.2266) |
Epoch 2 [100/383] | loss: 1.1222 (avg: 0.3308) | acc nat: 0.8281 (avg: 0.7191) | acc rob: 0.3125 (avg: 0.2560) |
Epoch 2 [200/383] | loss: 1.2651 (avg: 0.6453) | acc nat: 0.7188 (avg: 0.7280) | acc rob: 0.2344 (avg: 0.2633) |
Epoch 2 [300/383] | loss: 1.1526 (avg: 0.9540) | acc nat: 0.7812 (avg: 0.7353) | acc rob: 0.2656 (avg: 0.2658) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [04:38<00:00,  3.59it/s, acc_nat=0.863, acc_rob=0.527]


Epoch 98 [0/383] | loss: 0.1903 (avg: 0.0005) | acc nat: 0.9922 (avg: 0.9922) | acc rob: 0.8438 (avg: 0.8438) |
Epoch 98 [100/383] | loss: 0.1669 (avg: 0.0502) | acc nat: 1.0000 (avg: 0.9991) | acc rob: 0.8203 (avg: 0.8553) |
Epoch 98 [200/383] | loss: 0.1836 (avg: 0.0999) | acc nat: 1.0000 (avg: 0.9991) | acc rob: 0.8594 (avg: 0.8563) |
Epoch 98 [300/383] | loss: 0.1792 (avg: 0.1482) | acc nat: 1.0000 (avg: 0.9992) | acc rob: 0.8516 (avg: 0.8561) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [04:36<00:00,  3.62it/s, acc_nat=0.868, acc_rob=0.534]


Epoch 99 [0/383] | loss: 0.1518 (avg: 0.0004) | acc nat: 1.0000 (avg: 1.0000) | acc rob: 0.8906 (avg: 0.8906) |
Epoch 99 [100/383] | loss: 0.2199 (avg: 0.0494) | acc nat: 0.9922 (avg: 0.9989) | acc rob: 0.8594 (avg: 0.8560) |
Epoch 99 [200/383] | loss: 0.1848 (avg: 0.0976) | acc nat: 1.0000 (avg: 0.9991) | acc rob: 0.8125 (avg: 0.8569) |
Epoch 99 [300/383] | loss: 0.1661 (avg: 0.1452) | acc nat: 1.0000 (avg: 0.9989) | acc rob: 0.8828 (avg: 0.8589) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [04:30<00:00,  3.70it/s, acc_nat=0.871, acc_rob=0.528]
